In [ ]:
# STEP 1: Upload ZIP files separately
from google.colab import files
import zipfile, os

# Upload and extract your_images.zip
print("📤 Upload your_images.zip (positive samples)...")
uploaded = files.upload()
your_zip = list(uploaded.keys())[0]

with zipfile.ZipFile(your_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/your_images")

# Upload and extract other_images.zip
print("📤 Upload other_images.zip (negative samples)...")
uploaded = files.upload()
other_zip = list(uploaded.keys())[0]

with zipfile.ZipFile(other_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/other_images")

# STEP 2: Load MobileNetV2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

# STEP 3: Feature extractor
def extract_features(img_path, model, target_size=(224, 224)):
    try:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array, verbose=0)
        return features.flatten()
    except Exception as e:
        print(f"⚠ Error with {img_path}: {e}")
        return None

# STEP 4: Extract features from folders
def load_dataset_features(folder_path):
    features = []
    image_paths = []
    for root, _, files in os.walk(folder_path):
        for fname in sorted(files):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(root, fname)
                feat = extract_features(path, model)
                if feat is not None:
                    features.append(feat)
                    image_paths.append(path)
    print(f"✅ Loaded {len(features)} features from {folder_path}")
    return np.array(features), image_paths

your_features, your_paths = load_dataset_features("/content/your_images")
other_features, other_paths = load_dataset_features("/content/other_images")

# STEP 5: Upload test image
print("\n📤 Upload test image to predict...")
uploaded = files.upload()
test_image_path = list(uploaded.keys())[0]

test_feature = extract_features(test_image_path, model)

# STEP 6: Predict using similarity
def print_top_similarities(test_feat, reference_feats, reference_paths, label, k=5):
    similarities = cosine_similarity([test_feat], reference_feats)[0]
    top_k_idx = np.argsort(similarities)[-k:][::-1]
    print(f"\n🔍 Top {k} matches for {label}:")
    for i in top_k_idx:
        print(f"{reference_paths[i]} ➜ Similarity: {similarities[i]:.4f}")
    return similarities[top_k_idx].mean()

if test_feature is not None:
    print("\n--- Prediction Process ---")
    mean_sim_your = print_top_similarities(test_feature, your_features, your_paths, "POSITIVE (your_images)")
    mean_sim_other = print_top_similarities(test_feature, other_features, other_paths, "NEGATIVE (other_images)")

    margin = 0.03  # Use margin for decision

    print("\n--- Final Decision ---")
    print(f"📊 Mean similarity to POSITIVE: {mean_sim_your:.4f}")
    print(f"📊 Mean similarity to NEGATIVE: {mean_sim_other:.4f}")
    print(f"📊 Margin: {margin:.2f}")

    if mean_sim_your > mean_sim_other + margin:
        print("✅ Prediction: POSITIVE")
    else:
        print("❌ Prediction: NEGATIVE")
else:
    print("⚠ Could not extract features from test image.")

📤 Upload your_images.zip (positive samples)...


Saving your_images.zip to your_images.zip
📤 Upload other_images.zip (negative samples)...


Saving other_images.zip to other_images.zip


<ipython-input-1-84a177c51ef0>:28: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Loaded 60 features from /content/your_images
✅ Loaded 25 features from /content/other_images

📤 Upload test image to predict...


Saving Screenshot 2025-05-28 212823.png to Screenshot 2025-05-28 212823.png

--- Prediction Process ---

🔍 Top 5 matches for POSITIVE (your_images):
/content/your_images/your_images/Screenshot 2025-04-20 141930.png ➜ Similarity: 0.7042
/content/your_images/your_images/Screenshot 2025-04-20 141826.png ➜ Similarity: 0.7028
/content/your_images/your_images/Screenshot 2025-04-20 142111.png ➜ Similarity: 0.6864
/content/your_images/your_images/Screenshot 2025-04-20 142019.png ➜ Similarity: 0.6841
/content/your_images/your_images/Screenshot 2025-04-20 141639.png ➜ Similarity: 0.6746

🔍 Top 5 matches for NEGATIVE (other_images):
/content/other_images/other_images/Screenshot 2025-05-25 192600.png ➜ Similarity: 0.6755
/content/other_images/other_images/Screenshot 2025-05-25 192655.png ➜ Similarity: 0.6749
/content/other_images/other_images/Screenshot 2025-05-25 192402.png ➜ Similarity: 0.6624
/content/other_images/other_images/Screenshot 2025-05-25 192435.png ➜ Similarity: 0.6244
/content/other